In [15]:
# News Analysis Platform
# AI-driven financial news analysis using OpenAI and Perplexity APIs

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env")

from perplexity import Perplexity
from openai import OpenAI
import pandas as pd
from datetime import datetime

# --- Load API Keys ---
openai_api_key = os.getenv("OPENAI_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")

if not openai_api_key or not perplexity_api_key:
    raise EnvironmentError("Missing API keys in ../.env — please add OPENAI_API_KEY and PERPLEXITY_API_KEY")

# --- Initialize Clients ---
openai_client = OpenAI(api_key=openai_api_key)
pplx_client = Perplexity(api_key=perplexity_api_key)

print("OpenAI and Perplexity initialized successfully.")



OpenAI and Perplexity initialized successfully.


In [19]:
class PerplexitySearch:
    def __init__(self, perplexity_client, base_dir="../../outputs/portfolio1"):
        self.pplx_client = perplexity_client
        self.base_dir = base_dir
        self.analysis_dir = os.path.join(base_dir, "analysis")

    def search_perplexity(self, prompt, max_results=5):
        """
        Execute a Perplexity search given a text prompt.
        Returns a list of (title, url, snippet) dictionaries.
        """
        print(f"Searching Perplexity for: {prompt}")
        search = self.pplx_client.search.create(query=prompt, max_results=max_results)

        results = []
        for r in search.results:
            results.append({
                "title": r.title,
                "url": r.url,
                "snippet": r.snippet
            })
        return results

    def compile_perplexity_report(self, symbols, max_results=5, custom_prompt=None):
        report_lines = []
        for sym in symbols[:1]: 
            query = custom_prompt or f"Latest financial and market news for {sym} stock"
            try:
                news_items = self.search_perplexity(query, max_results=max_results)
                formatted_news = "\n".join(
                    [f"- {n['title']}: {n['url']}\n  {n['snippet']}" for n in news_items]
                )
                section = f"----- {sym} NEWS -----\n{formatted_news}\n"
                report_lines.append(section)
            except Exception as e:
                print(f"Error fetching news for {sym}: {e}")

        compiled_news = "\n".join(report_lines)

        # --- Build dated filename ---
        date_str = datetime.now().strftime("%Y-%m-%d")
        news_dir = os.path.join(self.analysis_dir, "news")
        os.makedirs(news_dir, exist_ok=True)

        news_filename = f"portfolio_news_digest_{date_str}.txt"
        news_path = os.path.join(news_dir, news_filename)

        # --- Save ---
        with open(news_path, "w") as f:
            f.write(compiled_news)

        print(f"Perplexity news digest saved to {news_path}")
        return compiled_news, news_path



In [20]:

class PortfolioNewsAnalyzer(PerplexitySearch):
    def __init__(self, perplexity_client, openai_client, base_dir="../../outputs/portfolio1", analysis_prompt=None):
        super().__init__(perplexity_client, base_dir)
        self.openai_client = openai_client
        self.analysis_prompt = analysis_prompt or self.default_prompt()

    def default_prompt(self):
        return """
You are a senior financial research analyst.

You have access to:
A quantitative portfolio report (performance and risk metrics)

Task:
- Tell me the main financial takeaways for this portfolio.
"""

    def load_portfolio_symbols(self):
        """Load stock symbols from performance.csv"""
        path = os.path.join(self.base_dir, "data", "performance.csv")
        df = pd.read_csv(path, index_col=0)
        symbols = df.index.tolist()
        print(f"Loaded {len(symbols)} symbols from portfolio: {symbols}")
        return symbols

    def call_openai(self, prompt_text, model="gpt-4o-mini", temperature=0.4):
        print("Calling OpenAI with custom prompt...")
        completion = self.openai_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a senior investment strategist."},
                {"role": "user", "content": prompt_text}
            ],
            temperature=temperature
        )
        return completion.choices[0].message.content

    def analyze_with_openai(self, news_digest, model="gpt-4o-mini", temperature=0.4):
        """
        Combine portfolio metrics + news digest, insert into prompt template,
        and analyze via OpenAI.
        """
        llm_report_path = os.path.join(self.analysis_dir, "portfolio_llm_report.txt")
        with open(llm_report_path, "r") as f:
            portfolio_text = f.read()

        prompt_text = f"""
{self.analysis_prompt}

===== PORTFOLIO REPORT =====
{portfolio_text}

"""
        analysis = self.call_openai(prompt_text, model=model, temperature=temperature)

        output_path = os.path.join(self.analysis_dir, "combined_news_portfolio_analysis.txt")
        with open(output_path, "w") as f:
            f.write(analysis)

        print(f"OpenAI analysis saved to {output_path}")
        return analysis, output_path


In [21]:
analyzer = PortfolioNewsAnalyzer(pplx_client, openai_client, analysis_prompt=None)
symbols = analyzer.load_portfolio_symbols()
news_digest, _ = analyzer.compile_perplexity_report(symbols)
analysis, _ = analyzer.analyze_with_openai(news_digest)

Loaded 17 symbols from portfolio: ['AAPL', 'MSFT', 'GOOGL', 'SPY', 'QQQ', 'VTI', 'TSLA', 'NVDA', 'JPM', 'BAC', 'VEA', 'VWO', 'JNJ', 'PFE', 'KO', 'PG', 'PORTFOLIO']
Searching Perplexity for: Latest financial and market news for AAPL stock
Perplexity news digest saved to ../../outputs/portfolio1/analysis/news/portfolio_news_digest_2025-10-19.txt
Calling OpenAI with custom prompt...
OpenAI analysis saved to ../../outputs/portfolio1/analysis/combined_news_portfolio_analysis.txt
